**PROYECTO COVID-19**

In [1]:
import cv2
import imgaug as ia
import imgaug.augmenters as iaa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib
import os
%matplotlib inline

Definimos la funcion para leer imagenes y aumentar el tamaño de nuestro conjunto de datos, utilizando:
- Ruido Gaussiano
- Volteando verticalmente

In [2]:
def lecturaImagenes(pathImagenes="Data/", dx=256, dy=256, noiseX=5, noiseY=10):
    if(os.path.exists(pathImagenes)):
        resultado = []
        listaImagenes = os.listdir(pathImagenes)

        #Realizar un data augmentation para aumentar los datos del dataset:
        gaussian_noise = iaa.AdditiveGaussianNoise(noiseX, noiseY)
        flip_vr=iaa.Fliplr(p=1.0)

        for imagen in listaImagenes:
            Xray = cv2.resize(cv2.imread(pathImagenes+imagen,0),(dx, dy))
            resultado.append(Xray)
            resultado.append(gaussian_noise.augment_image(Xray))
            resultado.append(flip_vr.augment_image(Xray))

        return resultado
    else:
        print("¡Error! El path especificado no ha sido encontrado")
        return None

Definimos una funcion para hacer el particionamiento de los datos en train, test y val

In [3]:
def particionamiento(listaDatos, train_percent=.6, validate_percent=.2, seed=None):
    np.random.seed(seed)
    listaTrain=[]
    listaTest=[]
    listaVal=[]
    y_train = []
    y_test = []
    y_val = []
    for i,lista in enumerate(listaDatos):
        lista = np.array(lista)
        perm = np.random.permutation(len(lista))
        m = len(lista)
        train_end = int(train_percent * m)
        val_end = int(validate_percent * m) + train_end
        train = lista[perm[:train_end]]
        val = lista[perm[train_end:val_end]]
        test = lista[perm[val_end:]]
        listaTrain.append(train)
        listaTest.append(test)
        listaVal.append(val)
        y_train.extend(list([i]*len(train)))
        y_test.extend(list([i]*len(test)))
        y_val.extend(list([i]*len(val)))
    
    return listaTrain, listaTest, listaVal, y_train, y_test, y_val

In [4]:
pathCovid = "Data/COVID-19/"
pathNormal = "Data/NORMAL/"
pathViral = "Data/Viral Pneumonia/" 

listaCOVID = lecturaImagenes(pathCovid)
listaNORMAL = lecturaImagenes(pathNormal)
listaVIRAL = lecturaImagenes(pathViral)


In [5]:
listaDatos = [listaCOVID, listaNORMAL, listaVIRAL] 
Train, Test, Val, y_train, y_test, y_val = particionamiento(listaDatos)